In [14]:
import gmaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Google developer API key
from Config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [15]:
#create cordinates for NY State map
coordinates = [(42.9915, -76.0718)]

In [17]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(zoom_level = 6.5, center = (42.9915, -76.0718), layout = figure_layout, map_type="HYBRID")

In [18]:
#introducing Vehicle data
vehicle_df = pd.read_csv("vehicle_data.csv.gz", compression="gzip")

#take care of NaN velue
vehicle_df = vehicle_df.dropna()

#creating dataframe for car's for selective years
vehicle_year_df = vehicle_df.loc[vehicle_df['model_year'].astype(int) <= 2009]

In [19]:
#calculating vehicle age till 2009 by model year 
veh_age = 2009 - vehicle_year_df['model_year'].astype(int)

#creating new data frame with new column for vehicle_age
new_vehicle_df = pd.DataFrame(vehicle_year_df)
new_vehicle_df['vehicle_age'] = veh_age

In [20]:
#introducing zip code coordinates
zip_df = pd.read_csv('../Se7en/us-zip-code-latitude-and-longitude.csv')
zip_df = zip_df.rename (columns = {"Zip" : "zip"})

#take care of NaN velue
zip_df = zip_df.dropna()

In [21]:
vehicle_age_2009_df = pd.merge(new_vehicle_df, zip_df, on=['zip','zip'], how="left")


#take care of NaN velue
vehicle_age_2009_df = vehicle_age_2009_df.dropna()
vehicle_age_2009_df

,vin,registration_class,zip,model_year,body_type,fuel_type,vehicle_age,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
0,999407G3573,PAS,12866,1976,SUBN,GAS,33,Saratoga Springs,NY,43.080399,-73.77038,-5.0,1.0
1,9992313,PAS,13316,1924,CONV,GAS,85,Camden,NY,43.377083,-75.74261,-5.0,1.0
2,998867,PAS,11501,1952,2DSD,GAS,57,Mineola,NY,40.747188,-73.63808,-5.0,1.0
3,9983692,PAS,10940,1936,2DSD,GAS,73,Middletown,NY,41.449364,-74.43951,-5.0,1.0
4,99782010525,PAS,11225,1978,2DSD,GAS,31,Brooklyn,NY,40.662892,-73.95509,-5.0,1.0


In [ ]:
#Plot Heatmap for vehicle age by zip code
locations = vehicle_age_2009_df[["Latitude", "Longitude"]]
vehicle_age = vehicle_age_2009_df["vehicle_age"].astype(float)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=vehicle_age, 
                                  dissipating=False, max_intensity=93,                                  
                                  point_radius=0.05)


# Add layer to the map
fig.add_layer(heat_layer)
fig